implementation of [universal style transfer via feature transforms by Li et al](https://arxiv.org/pdf/1705.08086.pdf)

In [1]:
# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
# make variables display whenever they are on their own line (not just the last line of a cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from utils import *

C:\Anaconda\envs\style\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Featureize images

In [16]:
compute_features('../val2017', out_layer_name='vgg_19/conv2/conv2_1', num_images=200)

INFO:tensorflow:Restoring parameters from vgg_19.ckpt


## Create graph

In [45]:
layer_to_sizes = {
    'conv1': (None, 224, 224, 64),
    'conv2': (None, 112, 112, 128)
}

tf.reset_default_graph()
sess = tf.Session()
images_ph = tf.placeholder('float', (None, 224, 224, 3))
features_ph = tf.placeholder('float', layer_to_sizes['conv2'])

In [46]:
dataset = make_precomputed_dataset('../val2017', 'conv2', num_images=200)

In [47]:
reconstructed_image = make_decoder(features_ph, [(3, 64), 'upsample', (3, 64), (3, 3)], sess)

In [48]:
loss = create_loss(images_ph, features_ph, reconstructed_image, 'vgg_19/conv2/conv2_1', sess)

INFO:tensorflow:Restoring parameters from vgg_19.ckpt


In [52]:
train(loss, dataset, images_ph, features_ph, sess, num_epochs=15, lr=1e-4)

[<tf.Variable 'decoder/conv_layer_0/filters:0' shape=(3, 3, 128, 64) dtype=float32_ref>, <tf.Variable 'decoder/conv_layer_0/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'decoder/conv_layer_2/filters:0' shape=(3, 3, 64, 64) dtype=float32_ref>, <tf.Variable 'decoder/conv_layer_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'decoder/conv_layer_3/filters:0' shape=(3, 3, 64, 3) dtype=float32_ref>, <tf.Variable 'decoder/conv_layer_3/biases:0' shape=(3,) dtype=float32_ref>]
[b'beta1_power_3' b'beta2_power_3' b'decoder/conv_layer_0/filters/Adam_6'
 b'decoder/conv_layer_0/filters/Adam_7'
 b'decoder/conv_layer_0/biases/Adam_6'
 b'decoder/conv_layer_0/biases/Adam_7'
 b'decoder/conv_layer_2/filters/Adam_6'
 b'decoder/conv_layer_2/filters/Adam_7'
 b'decoder/conv_layer_2/biases/Adam_6'
 b'decoder/conv_layer_2/biases/Adam_7'
 b'decoder/conv_layer_3/filters/Adam_6'
 b'decoder/conv_layer_3/filters/Adam_7'
 b'decoder/conv_layer_3/biases/Adam_6'
 b'decoder/conv_layer_3/biases/Adam_7']

In [57]:
saver = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'decoder'))
saver.save(sess, './decoder')

'./decoder'

## other crap

In [58]:
it = dataset.make_one_shot_iterator()

In [59]:
img = sess.run(tf.cast(it.get_next(), tf.uint8))

InvalidArgumentError: Shapes of all inputs must match: values[0].shape = [224,224,3] != values[1].shape = [112,112,128]
	 [[Node: Cast/x = Pack[N=2, T=DT_FLOAT, axis=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](IteratorGetNext_55/_1561, IteratorGetNext_55/_1563)]]
	 [[Node: Cast/_1565 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8_Cast", tensor_type=DT_UINT8, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Cast/x', defined at:
  File "C:\Anaconda\envs\style\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda\envs\style\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\envs\style\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda\envs\style\lib\site-packages\t

In [60]:
plt.imshow(img)

NameError: name 'img' is not defined

In [ ]:
decoded_img = sess.run(reconstructed_image, feed_dict={input_ph: img[np.newaxis,:,:,:]})

In [ ]:
decoded_img = np.squeeze(decoded_img)

In [ ]:
decoded_img[decoded_img > 255] = 255

In [ ]:
plt.imshow(decoded_img.astype(np.uint8))

In [ ]:
writer = tf.summary.FileWriter('summaries', sess.graph)

In [ ]:
sess.run(dataset.make_one_shot_iterator().get_next())

In [12]:
it = dataset.make_one_shot_iterator()
sess.run(it.get_next())[1].shape

(224, 224, 64)

In [41]:
# make_encoder(images_ph, sess)
writer = tf.summary.FileWriter('summaries', sess.graph)